In [1]:
import sys
import yaml
with open('..//secrets.yml', 'r') as file:
    secrets = yaml.safe_load(file)
sys.path.append(secrets['elo_proj_path'])

['c:\\Users\\Graeham\\Documents\\GITHUB_PROJECTS\\grieman.github.io\\site_resources',
 'c:\\Users\\Graeham\\AppData\\Local\\Programs\\Python\\Python310\\python310.zip',
 'c:\\Users\\Graeham\\AppData\\Local\\Programs\\Python\\Python310\\DLLs',
 'c:\\Users\\Graeham\\AppData\\Local\\Programs\\Python\\Python310\\lib',
 'c:\\Users\\Graeham\\AppData\\Local\\Programs\\Python\\Python310',
 '',
 'c:\\Users\\Graeham\\AppData\\Local\\Programs\\Python\\Python310\\lib\\site-packages',
 'c:\\Users\\Graeham\\AppData\\Local\\Programs\\Python\\Python310\\lib\\site-packages\\win32',
 'c:\\Users\\Graeham\\AppData\\Local\\Programs\\Python\\Python310\\lib\\site-packages\\win32\\lib',
 'c:\\Users\\Graeham\\AppData\\Local\\Programs\\Python\\Python310\\lib\\site-packages\\Pythonwin',
 'c:\\Users\\Graeham\\Documents\\GITHUB_PROJECTS\\Rugby_ELO']

In [2]:
from player_club_classes import team_elo, Player, Club, Match
from player_club_classes import GLOBAL_home_advantage, GLOBAL_score_factor, GLOBAL_k, GLOBAL_mov_score_add, GLOBAL_mov_score_mult, GLOBAL_mov_elo_mult, GLOBAL_mov_elo_add
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly
import pickle

from team_colors import team_color_dict
team_colors = pd.DataFrame(team_color_dict).T
team_colors.columns = ['Primary', 'Secondary']
team_colors = team_colors.rename_axis('Team').reset_index()

from plotly.validators.scatter.marker import SymbolValidator
raw_symbols = SymbolValidator().values

simple_symbols = [i for i in raw_symbols if str(i).isalpha()]

with open('../../Rugby_ELO/processed_data/playerbase.pickle', 'rb') as handle:
    playerbase = pickle.load(handle)
with open('../../Rugby_ELO/processed_data/matchlist.pickle', 'rb') as handle:
    matchlist = pickle.load(handle)
with open('../../Rugby_ELO/processed_data/teamlist.pickle', 'rb') as handle:
    teamlist = pickle.load(handle)

In [3]:
match_list = []
for _, match in matchlist.items():
    match_list.append({key:val for key, val in vars(match).items() if key not in ['home_team', 'away_team']})

match_df = pd.DataFrame(match_list)
match_df['week_num'] = match_df['date'].dt.isocalendar().week
match_df['month'] = match_df['date'].dt.month
match_df['year'] = match_df['date'].dt.year

In [4]:
month = match_df[(match_df.year == 2022) & (match_df.month == 5)]
closest_games = month.sort_values('lineup_spread',  key=abs).head()

sorted_pred_accuracy = month.assign(tmp=month["point_diff"] - month["lineup_spread"]).sort_values(by="tmp", key=abs)#.drop(columns="tmp")
best_preds = sorted_pred_accuracy.head()
worst_preds = sorted_pred_accuracy.tail()

biggest_upsets = month[(month.lineup_spread * month.point_diff) < 0].assign(tmp=month["point_diff"] - month["lineup_spread"]).sort_values(by="tmp", key=abs, ascending=False).drop(columns="tmp").head()
biggest_covers = month[((month.lineup_spread * month.point_diff) > 0)].assign(tmp=month["point_diff"] - month["lineup_spread"]).sort_values(by="tmp", key=abs, ascending=False)#.drop(columns="tmp").head()

In [5]:
player_elo_list = []
for player_name, player in playerbase.items():
    player_elo = player.elo_df
    player_elo['Full Name'] = player_name[0]
    player_elo['Unicode_ID'] = player_name[1]
    player_elo_list.append(player_elo)

player_elo = pd.concat(player_elo_list).reset_index(drop=True)
player_elo = pd.merge(player_elo, team_colors, on = 'Team', how = 'left')
player_elo['elo_change'] = player_elo.end_elo - player_elo.start_elo
player_elo['week_num'] = player_elo['Date'].dt.isocalendar().week
player_elo['month'] = player_elo['Date'].dt.month
player_elo['year'] = player_elo['Date'].dt.year

In [6]:
player_elo.tail()

,Unicode_ID,espn_id_num,Position,Position_Number,Minutes,Team,Date,Competition,Number,Home,...,Result,start_elo,end_elo,Full Name,Primary,Secondary,elo_change,week_num,month,year
453826,1101119510111511211095105100327311010597328497...,no_espn_id Inia Tabuavou,R,21.0,0.0,Racing 92,2022-05-21 11:15:00,Top 14 Orange 2022,21.0,False,...,Loss,100,100.0,Inia Tabuavou,white,#c4dcf4,0.0,20,5,2022
453827,1101119510111511211095105100326697112116105115...,no_espn_id Baptiste Fariscot,R,14.0,25.0,Biarritz Olympique,2022-05-21 11:15:00,Top 14 Orange 2022,22.0,True,...,Loss,100,99.86485,Baptiste Fariscot,#e2372a,white,-0.13515,20,5,2022
453828,1101119510111511211095105100327811110132681011...,no_espn_id Noe Della Schiava,R,19.0,13.0,La Rochelle,2022-05-21 11:15:00,Top 14 Orange 2022,19.0,True,...,Win,100,100.321213,Noe Della Schiava,#ebc931,black,0.321213,20,5,2022
453829,1101119510111511211095105100328397109327310810...,no_espn_id Sam Illo,R,3.0,25.0,Connacht,2022-05-21 12:00:00,United Rugby Championship 2022,18.0,True,...,Win,100,99.376925,Sam Illo,#5fb376,#616567,-0.623075,20,5,2022
453830,1101119510111511211095105100328610599116111114...,no_espn_id Victor Montgaillard,R,11.0,17.0,Perpignan,2022-05-22 15:05:00,Top 14 Orange 2022,16.0,False,...,Loss,100,99.720411,Victor Montgaillard,#ca2a16,#efc110,-0.279589,20,5,2022


In [7]:
# CHANGE
player_month = player_elo[(player_elo.year == 2022) & (player_elo.month == 5)]
oneplayer = player_month[(player_month.Unicode_ID == '5155544848') & (player_month['Full Name'] == 'Caleb Delany')].sort_values('Date')
oneplayer.iloc[-1].end_elo - oneplayer.iloc[0].start_elo

-2.554117441069806

In [11]:
def elo_change(df):
    return df.iloc[-1].end_elo, df.iloc[0].start_elo, df.iloc[-1].end_elo - df.iloc[0].start_elo

elo_changes = player_month.sort_values('Date').groupby(['Unicode_ID', 'Full Name', 'Team']).apply(elo_change).reset_index()
elo_changes[['end_elo', 'start_elo', 'elo_change']] = pd.DataFrame(elo_changes[0].tolist(), index=elo_changes.index)
elo_changes.sort_values('elo_change')

,Unicode_ID,Full Name,Team,0,end_elo,start_elo,elo_change
576,5051515156,James Blackwell,Hurricanes,"(100.88391371265058, 106.42990302317858, -5.54...",100.883914,106.429903,-5.545989
826,5055485550,Salesi Rayasi,Hurricanes,"(110.70188334521939, 115.67822631062886, -4.97...",110.701883,115.678226,-4.976343
1070,5149525654,Quinn Tupaea,Chiefs,"(113.26707089934938, 118.08496318931884, -4.81...",113.267071,118.084963,-4.817892
267,5048485452,Ardie Savea,Hurricanes,"(118.33886733853684, 122.95562106235826, -4.61...",118.338867,122.955621,-4.616754
589,5051515748,Josh Ioane,Chiefs,"(98.22976447566921, 102.77928260508381, -4.549...",98.229764,102.779283,-4.549518
...,...,...,...,...,...,...,...
406,5049535052,Mattia Bellini,Benetton Treviso,"(96.79422627952788, 92.30762615721996, 4.48660...",96.794226,92.307626,4.486600
745,5053505254,Rhyno Smith,Benetton Treviso,"(99.09602275738854, 94.60942263508062, 4.48660...",99.096023,94.609423,4.486600
244,4957555455,Juan Ignacio Brex,Benetton Treviso,"(98.82188475913395, 94.33528463682603, 4.48660...",98.821885,94.335285,4.486600
408,5049535155,Federico Ruzza,Benetton Treviso,"(102.4315904715746, 97.94499034926667, 4.48660...",102.431590,97.944990,4.486600
